In [15]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
%matplotlib inline

# annoying warnings
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [16]:
def filtering_features_pipeline(data, target_file, temp_increase_vars):
    '''
    Function to run all of our preprocessing functionalities, with a few different configuration options.
    Will save a new dataframe as csv file in a specified address
    
    INPUTS:
    data: a pandas dataframe containing the raw data
    target_file: a file name and address to save the preprocessed data frames. expects .csv extension
    temp_increase_vars: a boolean value (True or False) on whether or not to create the binary temperature increase variables
    
    OUTPUTS:
    data: a data frame with the preprocessed data
    file saved... the preprocessed data frame saved as a csv file at the specified file extension
    '''
    
    #Add campaign duration and campaignID columns
    #Need to convert column to date_time type
    data['Date.Time'] = pd.to_datetime(data['Date.Time'])

    #Loop through and identify consecutive dates. Create a list of campaign IDs to then create a column from.
    campaign_num = 1
    campaign_id_list = [1]
    for index in range(1,len(data['Date.Time'])):
        if (data['Date.Time'][index] - data['Date.Time'][index-1]).days < 1:
            campaign_id_list.append(campaign_num)
        else:
            campaign_num += 1
            campaign_id_list.append(campaign_num)

    #Create new column of campaign IDs:
    data['CampaignID'] = campaign_id_list

    #Loop through and calculate hours in campaign
    duration_list = [0]

    for index in range(1,len(data['Date.Time'])):
        campaign_num = data['CampaignID'][index]
        campaign_start = min(data['Date.Time'][data['CampaignID'] == campaign_num])
        hrs_since_start = (data['Date.Time'][index] - campaign_start).delta*2.77778e-13
        duration_list.append(hrs_since_start)

    #Create new column for campaign hours
    data['Campaign.Hrs'] = duration_list
    
    dates = data['Date.Time']
    data.drop(['Unnamed: 0', 'Date.Time'], axis = 1, inplace = True)
    
    data.drop(['Pressure'], axis = 1, inplace = True)
    
    #delete rows where Main_Mass_Flow =0
    rows_to_delete = np.where(data.Main_Mass_Flow == 0)[0]
    # also add 337, 2533
    rows_to_delete = np.sort(np.concatenate((rows_to_delete, np.array([337, 2533])), axis = 0))

    data.drop(rows_to_delete, axis = 0, inplace = True)
    
    # replace T_Zone_1 zero vals with median
    new_zero_val_T = np.median(data.T_Zone_1.iloc[np.where(data.T_Zone_1 > 50)])
    data.T_Zone_1.iloc[np.where(data.T_Zone_1 <= 50)] = new_zero_val_T

    # replace Blending vals with median
    new_zero_val_B = np.median(data.Blending.iloc[np.where(data.Blending > 20)])
    data.Blending.iloc[np.where(data.Blending <= 20)] = new_zero_val_B
    
    data = data[data.Main_Mass_Flow > 15000]
    
    if temp_increase_vars:
        for i in range(1,10):
            new_colname = "T_Increase_" + str(i)
            colname_i = "T_Zone_" + str(i)
            colname_inext = "T_Zone_" + str(i+1)
            vals = (data[colname_i] <= data[colname_inext]).astype(int)
            data[new_colname] = vals
   
    data.to_csv(target_file)
    return data

        
    


In [1]:
def scale_training(data, scaler):
    '''
    A function that will peform scaling on training data given that all features are already created.
    
    INPUTS:
    data: a pandas dataframe to be scaled
    scaler: a scaling protocol: "standard" or "minmax"
    
    RETURNS:
    scaler: a scaler object that can be used to scale testing data
    scaled_features: a scaled pandas dataframe object
    '''
    if scaler == "standard":
        scaled_colnames = data.columns
        scaled_features = data.copy()
        features = scaled_features[scaled_colnames]
        scaler = StandardScaler().fit(features.values)
        features = scaler.transform(features.values)
        scaled_features[scaled_colnames] = features
        
    elif scaler == "minmax":
        scaled_colnames = data.columns
        scaled_features = data.copy()
        features = scaled_features[scaled_colnames]
        scaler = MinMaxScaler().fit(features.values)
        features = scaler.transform(features.values)
        scaled_features[scaled_colnames] = features
    
    else:
        raise ValueError(scaler)
        
    return scaler, scaled_features

In [13]:
data = pd.read_csv("anonymized_SAP_data.csv")

In [27]:
filtering_features_pipeline(data, 'no_tempinc.csv', False)

In [16]:
filtering_features_pipeline(data, 'tempinc.csv', True)

In [18]:
data_files = ['no_tempinc.csv', 'tempinc.csv']

In [20]:
scalers = ['standard','minmax']

In [35]:
for file in data_files:
    data = pd.read_csv(str('./p_data/')+file)
    for scaler in scalers:
        sc, temp_df = scale_training(data,scaler)
        file_name = file[:-4]+ str('_') + scaler + str('.csv')
        temp_df.to_csv('./p_data/'+file_name)
        